In [1]:
%AddJar https://repo.eclipse.org/content/repositories/paho-releases/org/eclipse/paho/org.eclipse.paho.client.mqttv3/1.0.2/org.eclipse.paho.client.mqttv3-1.0.2.jar -f
%AddJar http://central.maven.org/maven2/com/google/code/gson/gson/2.2.4/gson-2.2.4.jar -f
%AddJar https://github.com/sathipal/spark-streaming-mqtt-with-security_2.10-1.3.0/releases/download/0.0.1/spark-streaming-mqtt-security_2.10-1.3.0-0.0.1.jar -f

Starting download from https://repo.eclipse.org/content/repositories/paho-releases/org/eclipse/paho/org.eclipse.paho.client.mqttv3/1.0.2/org.eclipse.paho.client.mqttv3-1.0.2.jar
Finished download of org.eclipse.paho.client.mqttv3-1.0.2.jar


Waiting for a Spark session to start...

Starting download from http://central.maven.org/maven2/com/google/code/gson/gson/2.2.4/gson-2.2.4.jar
Finished download of gson-2.2.4.jar
Starting download from https://github.com/sathipal/spark-streaming-mqtt-with-security_2.10-1.3.0/releases/download/0.0.1/spark-streaming-mqtt-security_2.10-1.3.0-0.0.1.jar
Finished download of spark-streaming-mqtt-security_2.10-1.3.0-0.0.1.jar


In [2]:
import org.eclipse.paho.client.mqttv3._
import org.apache.spark.SparkContext
import org.apache.spark.storage.StorageLevel
import org.apache.spark.streaming.{Seconds, StreamingContext}
import org.apache.spark.streaming.mqtt._
import org.apache.spark.SparkConf

In [3]:
val ssc = new StreamingContext(sc.asInstanceOf[SparkContext], Seconds(1))

val lines = MQTTUtils.createStream(ssc, // Spark Streaming Context
"ssl://3etv6p.messaging.internetofthings.ibmcloud.com:8883", // Watson IoT Platform URL
"iot-2/type/+/id/+/evt/+/fmt/+", // MQTT topic to receive the events
"a:3etv6p:random", // Unique ID of the application
"a-3etv6p-7icb4uf5fh", // API-Key
"wgbXafCw*!eR3x?lDm") // Auth-Token

In [4]:
import java.util.Map.Entry
import com.google.gson.JsonElement
import com.google.gson.JsonParser
import java.util.Set
import collection.mutable.HashMap

In [5]:
val deviceMappedLines = lines.map(x => ((x.split(" ", 2)(0)).split("/")(4), x.split(" ", 2)(1)))

In [6]:
// Map the Json payload into scala map
val jsonLines = deviceMappedLines.map(x => {
 var dataMap:scala.collection.mutable.Map[String, Any] = scala.collection.mutable.Map()
 val payload = new JsonParser().parse(x._2).getAsJsonObject()
 var deviceObject = payload
 val setObj = deviceObject.entrySet()
 val itr = setObj.iterator()
 while(itr.hasNext()) {
 val entry = itr.next();
 try {
 dataMap.put(entry.getKey(), entry.getValue().getAsDouble())
 } catch {
 case e: Exception => dataMap.put(entry.getKey(), entry.getValue().getAsString())
 }
 }
 (x._1, dataMap)
})

In [7]:
val countEvMap = new HashMap[Tuple2[Int,Int],Int]()

In [8]:
/**
 * Create a simple threshold rule. If coordinates are too large, alert
 */
 
 val threasholdCrossedLines = jsonLines.filter(
 x => {
 var status = false;
 val lat = x._2.get("latitude").toString.toDouble
 val lon = x._2.get("longitude").toString.toDouble
 if(lat != null && lon != null) {
	 /* example */
     val latInt = 100.0*lat;
     val lonInt = 100.0*lon;
     val t = new Tuple2[Int,Int](latInt.toInt,lonInt.toInt)
     countEvMap.put(t, countEvMap.get(t).getOrElse(0) + 1)
    status = true;

 }
     status
})

Name: org.apache.spark.SparkException
Message: Task not serializable
StackTrace:   at org.apache.spark.util.ClosureCleaner$.ensureSerializable(ClosureCleaner.scala:298)
  at org.apache.spark.util.ClosureCleaner$.org$apache$spark$util$ClosureCleaner$$clean(ClosureCleaner.scala:288)
  at org.apache.spark.util.ClosureCleaner$.clean(ClosureCleaner.scala:108)
  at org.apache.spark.SparkContext.clean(SparkContext.scala:2107)
  at org.apache.spark.streaming.dstream.DStream$$anonfun$filter$1.apply(DStream.scala:559)
  at org.apache.spark.streaming.dstream.DStream$$anonfun$filter$1.apply(DStream.scala:559)
  at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
  at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
  at org.apache.spark.SparkContext.withScope(SparkContext.scala:709)
  at org.apache.spark.streaming.StreamingContext.withScope(StreamingContext.scala:264)
  at org.apache.spark.streaming.dstream.DStream.filter(DStream.scala:55

In [9]:
threasholdCrossedLines.print()

ssc.start()
ssc.awaitTermination()

Name: Unknown Error
Message: lastException: Throwable = null
<console>:41: error: not found: value threasholdCrossedLines
       threasholdCrossedLines.print()
       ^
StackTrace: 